# 📌 상품 정보 크롤링(롯데온)

### ✏️ 하위 카테고리 별 상품 링크 저장

In [13]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# 주소 담을 리스트
page_list = []

# small_categories.txt 파일 읽어오기
small_categories = "C:\SSAFY\자율프로젝트\자율크롤링\small_categories.txt"
# small_categories = "D:\SSAFY\자율\크롤링\자율크롤링\small_categories.txt"
f = open(small_categories, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    category_url = strings[1]
    print(category_uid)
    print(category_url)
    
    ## 크롤링
    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(category_url)
    web_driver.implicitly_wait(5)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    link_list = soup.find_all('a', class_='srchGridProductUnitLink',href=True)
    
    # print(link_list)
    # print(len(link_list))

#     목록에서 link 얻어오기 (2개씩 얻어와지기때문에 짝수번은 날려보내기)
    for index, list in enumerate(link_list):
        if index %2 == 0:
            page_list.append(category_uid + " " + list['href'])
        if index >= 39:
            break
            
time.sleep(5)

    # page_list 저장하기 
# fo = open("D:\SSAFY\자율\크롤링\자율크롤링\products.txt", "w", encoding="UTF8")
fo = open("C:\SSAFY\자율프로젝트\자율크롤링\products.txt", "w", encoding="UTF8")
for page in page_list:
    fo.write(page)
    fo.write("\n")
fo.close
    
print(len(page_list))

    
f.close


    



    


1
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=FC01200000&login=Y&mallId=1
2
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01260000&mallId=1
3
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&u9=navigate&u8=FC01080000&login=Y&mallId=1
4
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01040000&mallId=1
5
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC01220000&mallId=1
6
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC02080000&mallId=1
7
https://www.lotteon.com/search/render/render.ecn?render=nqapi&platform=pc&collection_id=9&login=Y&u9=navigate&u8=FC02090000&mallId=1
8
https://www.lotteon.com/search/render/render.ecn?rend

<function TextIOWrapper.close()>

### ✏️ 상품 정보 DB에 저장 (product, brand, picture)

In [18]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn

# small_categories.txt 파일 읽어오기
# products = "D:\SSAFY\자율\크롤링\자율크롤링\products.txt"
# products = "C:\SSAFY\자율프로젝트\자율크롤링\products.txt"
products = "C:\SSAFY\자율프로젝트\자율크롤링\products_temp.txt"
f = open(products, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    url = strings[1]
    print(category_uid)
    print(url)
    print("------------------------")

    # 임시 url
    # temp_url= "https://www.lotteon.com/p/product/LO1987818462?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC01200000&clickId=C33295679634"
#     temp_url = "https://www.lotteon.com/p/product/LO1566256308?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC04020000&clickId=C33403790992"

    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(url)
    web_driver.implicitly_wait(3)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 데이터 찾기 
    product_name = soup.find('div', class_='productName').text
    des_picture = soup.find('iframe', class_='m2-prd-frame')['src']

    status = '새상품'

    ## 할인율 없으면
    if soup.find_all("span", {"class": "discountRate"}):
        discount_rate = soup.find('span', class_='discountRate').text.replace("할인율","").replace(" ","").replace("%","")
        product_price = soup.find('div', class_='listPrice').select_one(":nth-child(2)").text.replace(" ","").replace("\n","").replace(",","")
    else:
        discount_rate = 0
        product_price = soup.find('div', class_='price').text.replace(" ","").replace("\n","").replace("할인가","").replace("원","").replace(",","")

    # int값으로 변환
    product_price = int(product_price)
    discount_rate = int(discount_rate)

    # 리뷰가 있는지 확인
    if soup.find_all("a", {"class": "staring"}):
        review_rating = soup.find('a', class_='staring').em.text # 5.0
    else:
        review_rating = 0;
    delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text

    # 배송비 무료 > int 변환
    val = "무료" in delivery_fee
    if val:
        delivery_fee = 0
    else:
        delivery_fee = 3000



    # 이미지(썸네일) 
    picture_list = soup.find('div', class_='productVisualThumbs')
#   rep_img =  picture_list.find('img')['src']
    rep_img = picture_list.find('img')['src']
#     print("rep_img",rep_img)
    rep_img_list =  picture_list.find_all('img')
#     for img in rep_img_list:
#         print(img['src'])


    ############### 브랜드 ################
    brand_uid = 0
    brand_isAlready = True
    if soup.find_all("div", {"class": "brand"}):
        brand_name = soup.find("div", {"class": "brand"}).text
        brand_url = soup.find("div", {"class": "brandImage"}).select_one(":nth-child(1)")['src']
        print(brand_name)
        # 해당 브랜드를 가지고 있는지 확인
        sql = "SELECT uid FROM brands WHERE name = %s"
        with connectiondb() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, brand_name)
                result = cur.fetchone()
                if result is None:
                    print("table에 브랜드없음")
                    brand_isAlready = False
                else: 
                    print("table에 브랜드있음")
                    brand_isAlready = True
                    brand_uid = result[0]
    else:
        print("브랜드없는상품")
        brand_uid =  1 # 브랜드 없음 


    if(brand_isAlready is False):
        print("브랜드생성")
        # 없으면 새로 만들고 uid 값 가져오기
        sql = "INSERT INTO brands (img_url, name) VALUES (%s, %s)"
        with connectiondb() as conn:
            with conn.cursor() as cur:
                cur.execute(sql, (brand_url, brand_name))
                brand_uid = cur.lastrowid
                conn.commit()
    print("@@brand insert@@")

    ############### product insert ################
    # 형변환
    category_uid = int(category_uid)
    review_rating = float(review_rating)
    
    
#     print("delivery_fee",delivery_fee)
#     print("discount_rate",discount_rate)
#     print("price",product_price)
#     print("name",product_name)
#     print("description_img",des_picture)
#     print("rep_img",rep_img)
#     print("review_rate",review_rating)
#     print("status",status)
#     print("brand_uid",brand_uid)
#     print("small_categories_uid", category_uid)
    
    
#     print(type(delivery_fee))
#     print(type(des_picture))
#     print(type(discount_rate))
#     print(type(product_name))
#     print(type(product_price))
#     print(type(rep_img))
#     print(type(review_rating))
#     print(type(status))
#     print(type(brand_uid))
#     print(type(category_uid))
    

    sql = "INSERT INTO products (delivery_fee, description_img, discount_rate, name, price, rep_img, review_rate, status, brands_uid, small_categories_uid) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (delivery_fee, des_picture, discount_rate, product_name, product_price, rep_img, review_rating, status, brand_uid, category_uid))
            product_uid = cur.lastrowid
            conn.commit()
            
    print("@@product insert@@")


    ############### 이미지 insert ################
    ## insert 이후 가장 최근 product uid 데려와서 상품 사진 테이블에 insert 하기 
    min_length = min(len(rep_img_list),3)
#     print("min_length",min_length)
    
    for i in range(min_length):
        product_thumbnail = rep_img_list[i]['src']
#         print("product_thumbnail",product_thumbnail)
    
        sql = "INSERT INTO product_pictures (img_url,products_uid) VALUES (%s,%s)"
        with connectiondb() as conn: 
            with conn.cursor() as cur:
                cur.execute(sql, (product_thumbnail, product_uid))
                conn.commit()
    print("@@image insert@@")
    ############### 옵션 insert ################
    # 워크벤치에서 따로 
    
    print("====================================================")

    

f.close


25
https://www.lotteon.com/p/product/LO1645491725?sitmNo=LO1645491725_1645491726&mall_no=1&dp_infw_cd=CASfc04020000
------------------------
케라셀
table에 브랜드없음
브랜드생성
@@brand insert@@
@@product insert@@
@@image insert@@
25
https://www.lotteon.com/p/product/LO1905915980?sitmNo=LO1905915980_1905915981&mall_no=1&dp_infw_cd=CASFC04020000
------------------------
공식 브랜드관베스트 브랜드에뛰드
table에 브랜드있음
@@brand insert@@
@@product insert@@
@@image insert@@
25
https://www.lotteon.com/p/product/LO1819777541?sitmNo=LO1819777541_1819777543&mall_no=1&dp_infw_cd=CASFC04020000
------------------------
브랜드없는상품
@@brand insert@@
@@product insert@@
@@image insert@@


<function TextIOWrapper.close()>

### ✏️ 옵션 넣기

In [103]:
import requests
import re
import os
import pymysql


def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn


## 1. 과일
sql = "select uid from wkdbf.products where small_categories_uid = 38"
with connectiondb() as conn: 
    with conn.cursor() as cur:
        cur.execute(sql)
        result = cur.fetchall()

        
for temp in result:
    print(temp[0])

    sql = "insert into wkdbf.product_options (name, value, products_uid) values (%s,%s,%s);"
    with connectiondb() as conn: 
        with conn.cursor() as cur:
            cur.execute(sql, ('color', '블랙', temp[0]))
            conn.commit()

512


### ✏️ 추가 카테고리 데이터 넣기(카테고리당 한개)  

In [7]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn





# 임시 url
temp_url= "https://www.lotteon.com/p/product/LO1532037525?areaCode=AD_CPC&entryPoint=ad&dp_infw_cd=CASFC08070000&clickId=C33458657195"

category_uid = 39


webdriver_options=webdriver.ChromeOptions()
webdriver_options.add_argument("--start-maximized")
web_driver = webdriver.Chrome(options=webdriver_options)
web_driver.get(temp_url)
web_driver.implicitly_wait(3)
html = web_driver.page_source
soup = BeautifulSoup(html, 'html.parser')


    # 데이터 찾기 
product_name = soup.find('div', class_='productName').text
des_picture = soup.find('iframe', class_='m2-prd-frame')['src']

status = '새상품'

    ## 할인율 없으면
if soup.find_all("span", {"class": "discountRate"}):
    discount_rate = soup.find('span', class_='discountRate').text.replace("할인율","").replace(" ","").replace("%","")
    product_price = soup.find('div', class_='listPrice').select_one(":nth-child(2)").text.replace(" ","").replace("\n","").replace(",","")
else:
    discount_rate = 0
    product_price = soup.find('div', class_='price').text.replace(" ","").replace("\n","").replace("할인가","").replace("원","").replace(",","")

    # int값으로 변환
product_price = int(product_price)
discount_rate = int(discount_rate)

    # 리뷰가 있는지 확인
if soup.find_all("a", {"class": "staring"}):
    review_rating = soup.find('a', class_='staring').em.text # 5.0
else:
    review_rating = 0;
delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text

    # 배송비 무료 > int 변환
val = "무료" in delivery_fee
if val:
    delivery_fee = 0
else:
    delivery_fee = 3000



    # 이미지(썸네일) 
picture_list = soup.find('div', class_='productVisualThumbs')
#   rep_img =  picture_list.find('img')['src']
rep_img = picture_list
#     print("rep_img",rep_img)
rep_img_list =  picture_list.find_all('img')
for img in rep_img_list:
    print(img['src'])
main_img = soup.find('div', class_='largeImgSlide').find('img')['src']
print("메인썸네일", main_img)

    ############### 브랜드 ################
brand_uid = 0
brand_isAlready = True
if soup.find_all("div", {"class": "brand"}):
    brand_name = soup.find("div", {"class": "brand"}).text
    brand_url = soup.find("div", {"class": "brandImage"}).select_one(":nth-child(1)")['src']
    print(brand_name)
        # 해당 브랜드를 가지고 있는지 확인
    sql = "SELECT uid FROM brands WHERE name = %s"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, brand_name)
            result = cur.fetchone()
            if result is None:
                print("table에 브랜드없음")
                brand_isAlready = False
            else: 
                print("table에 브랜드있음")
                brand_isAlready = True
                brand_uid = result[0]
else:
    print("브랜드없는상품")
    brand_uid =  1 # 브랜드 없음 


if(brand_isAlready is False):
    print("브랜드생성")
        # 없으면 새로 만들고 uid 값 가져오기
    sql = "INSERT INTO brands (img_url, name) VALUES (%s, %s)"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (brand_url, brand_name))
            brand_uid = cur.lastrowid
            conn.commit()
print("@@brand insert@@")

    ############### product insert ################
    # 형변환
category_uid = int(category_uid)
review_rating = float(review_rating)


print("delivery_fee",delivery_fee)
print("discount_rate",discount_rate)
print("price",product_price)
print("name",product_name)
print("description_img",des_picture)
print("rep_img",rep_img)
print("review_rate",review_rating)
print("status",status)
print("brand_uid",brand_uid)
print("small_categories_uid", category_uid)
    

sql = "INSERT INTO products (delivery_fee, description_img, discount_rate, name, price, rep_img, review_rate, status, brands_uid, small_categories_uid) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
# with connectiondb() as conn:
#     with conn.cursor() as cur:
#         cur.execute(sql, (delivery_fee, des_picture, discount_rate, product_name, product_price, rep_img, review_rating, status, brand_uid, category_uid))
#         product_uid = cur.lastrowid
#         conn.commit()
            
print("@@product insert@@")


    ############### 이미지 insert ################
    ## insert 이후 가장 최근 product uid 데려와서 상품 사진 테이블에 insert 하기 
min_length = min(len(rep_img_list),3)
#     print("min_length",min_length)
    
for i in range(min_length):
    product_thumbnail = rep_img_list[i]['src']
#         print("product_thumbnail",product_thumbnail)
    
#     sql = "INSERT INTO product_pictures (img_url,products_uid) VALUES (%s,%s)"
#     with connectiondb() as conn: 
#         with conn.cursor() as cur:
#             cur.execute(sql, (product_thumbnail, product_uid))
#             conn.commit()
print("@@image insert@@")
    ############### 옵션 insert ################
    # 워크벤치에서 따로 
    
print("====================================================")



https://contents.lotteon.com/itemimage/_v125620/LO/15/32/03/75/25/_1/53/20/37/52/6/LO1532037525_1532037526_1.jpg/dims/resizef/67x67
메인썸네일 https://contents.lotteon.com/itemimage/_v125620/LO/15/32/03/75/25/_1/53/20/37/52/6/LO1532037525_1532037526_1.jpg/dims/resizef/554X554
카르닉
table에 브랜드있음
@@brand insert@@
delivery_fee 0
discount_rate 0
price 47900
name 1+1 카르닉 폴딩 캠핑의자 (블랙) 야외 낚시 폴딩의자
description_img https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F15%2F32%2F03%2F75%2F25%2FDSCRP_LO1532037525
rep_img <div class="swiper-container productVisualThumbs swiper-container-initialized swiper-container-vertical" data-v-02fae411=""> <div class="swiper-wrapper" style="transition-duration: 0ms; transform: translate3d(0px, 159px, 0px);"><div class="swiper-slide swiper-slide-active" data-v-02fae411="" style="margin-bottom: 10px;"><div class="imgWrap" data-cmpnt-name="pdd_thumb_preview_img" data-cmpnt-typ="pdd_basicinfo" data-gtm-vis-first-on-screen-11901096_2002="1437" data-gt

### ✏️ 메인 이미지 사진 변경하기 

In [10]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import pymysql

# DB 연결용
# conn = pymysql.connect(host='k7e201.p.ssafy.io',
#                        user='abuyz7e201',
#                        password='a-buy-z-e201',
#                        db='wkdbf',
#                        charset='utf8')

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn

products = "C:\SSAFY\자율프로젝트\자율크롤링\products_temp.txt"
f = open(products, "r", encoding="UTF8")
    
# 22부터 513까지 
uid = 419


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    url = strings[1]
    print(category_uid)
    print(url)
    print("@@@@@@@@@@uid : ", uid)
    print("------------------------")

    sql = "select * from wkdbf.products where uid = %s"
    with connectiondb() as conn: 
        with conn.cursor() as cur:
            cur.execute(sql, uid)
            result = cur.fetchall()
    print(result)
    

    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    webdriver_options.add_argument("headless")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(url)
    web_driver.implicitly_wait(3)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 메인 상품 이미지
    main_img = soup.find('div', class_='largeImgSlide').find('img')['src']
    print("@@@@@@@@@@메인썸네일 : ", main_img)


    sql = "UPDATE wkdbf.products SET rep_img=%s WHERE uid=%s"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (main_img, uid))
            conn.commit()
            
    uid += 1
    if uid == 25 or uid == 75:
        uid += 1
        
    if(uid == 505):
        uid += 2
    
    print("===============================================")
    
    
    # driver 종료
#     webdriver.quit()

f.close

20
https://www.lotteon.com/p/product/LE1212352328?sitmNo=LE1212352328_1270137092&mall_no=1&dp_infw_cd=CASFC12040000
@@@@@@@@@@uid :  419
------------------------
((419, datetime.datetime(2022, 11, 3, 2, 6, 2), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLE%2F12%2F12%2F35%2F23%2F28%2FDSCRP_LE1212352328', 0, '블랙 시크 드레스 자켓', 87000, 'https://contents.lotteon.com/itemimage/_v145827/LE/12/12/35/23/28/_1/27/01/37/09/2/LE1212352328_1270137092_1.jpg/dims/resizef/67x67', 0.0, '새상품', 151, 20, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v093813/LE/12/12/35/23/28/_1/27/01/37/09/2/LE1212352328_1270137092_1.jpg/dims/resizef/554X554
20
https://www.lotteon.com/p/product/LO1584060924?sitmNo=LO1584060924_1584060925&mall_no=1&dp_infw_cd=CASFC12040000
@@@@@@@@@@uid :  420
------------------------
((420, datetime.datetime(2022, 11, 3, 2, 6, 11), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F15%2F84%2F06%2F09%2F24%2FD

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/LO/17/18/87/25/67/_1/71/88/72/56/8/LO1718872567_1718872568_3.jpg/dims/resizef/554X554
21
https://displayad.lotteon.com/cadClick?x=LHgFFJfDpYXzIP1TBOBUMhpgeB4euLEtMSB9dikoQtW3ZXcxIdf1I9U1r9sDZXlYV0KBS%2FQBDRFc%2BUit7xKMUgJuia6eeW3zJXS3aWMv%2FggUDbfpUTnSkShhomRl0McYwDpoTRryHg4Uopd9jbE1QhxrUSeS%2F2YC7kyMi0sMdPKDLPrk4JYY%2BMELN5khUodEmurlbt4fWyrUkzTL8uC%2Bx1AaITqH%2FlzWDtQpxTb%2BbQ3TYnLLASEKf2yh2m7YB93xy9SZ%2FuMksUtfmlbgN55eLQ%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1584248150%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26dp_infw_cd%3DCASFC04090000
@@@@@@@@@@uid :  429
------------------------
((429, datetime.datetime(2022, 11, 3, 2, 7, 42), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F15%2F84%2F24%2F81%2F50%2FDSCRP_LO1584248150', 0, '(1+1 행사중, 파격 가격인하) 동충하초 시즌2 크림 50ml, 국제저널 SCI 검증, 피부탄력을 잃을 때', 17000, 'https://contents.lotteon.com/itemimage/_v055505/LO/15/84/24/81/50/_1/58/42/48/1

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v152145/LO/19/88/50/03/84/_1/98/85/00/38/5/LO1988500384_1988500385_1.jpg/dims/resizef/554X554
21
https://www.lotteon.com/p/product/LO1955894916?sitmNo=LO1955894916_1955894917&mall_no=1&dp_infw_cd=CASFC04090000
@@@@@@@@@@uid :  440
------------------------
((440, datetime.datetime(2022, 11, 3, 2, 9, 19), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F19%2F55%2F89%2F49%2F16%2FDSCRP_LO1955894916', 15, '[패드케이스 증정] 캐롯 카로틴 카밍 워터 패드 60매 250gX2', 52000, 'https://contents.lotteon.com/itemimage/_v085714/LO/19/55/89/49/16/_1/95/58/94/91/7/LO1955894916_1955894917_1.jpg/dims/resizef/67x67', 4.9, '새상품', 163, 21, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v085714/LO/19/55/89/49/16/_1/95/58/94/91/7/LO1955894916_1955894917_1.jpg/dims/resizef/554X554
21
https://www.lotteon.com/p/product/LO1854696853?sitmNo=LO1854696853_1854696854&mall_no=1&dp_infw_cd=CASFC04090000
@@@@@@@@@@uid :  441
--

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v172736/LO/16/47/69/36/93/_1/64/76/93/69/4/LO1647693693_1647693694_1.jpg/dims/resizef/554X554
22
https://displayad.lotteon.com/cadClick?x=H3zeXuJQBe2cxXYlfEonWprXziLV4GeuYLTxyoWR237aOTsrH0FN89raN2jfOOCSz7e%2BtQz8sDQBT3csut6dzoS871hDdcEV7pty7SETpFNbTegmjfgPF01knhsbQZRoU2YjcHb93wfQPCSX330cwvReai0nrtoMK6WdeaYEhiX5cUOIiPVczM2qhSruda3fAu0mk1CgYc3mpTqGi1GN0bjEygWeLTsAA1X2Q7XHlGF96tc2b%2FHeADdrSom01qQk1zEj%2BWI9JXxXCAya2KQDTw%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1647815987%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26dp_infw_cd%3DCASFC04110000
@@@@@@@@@@uid :  449
------------------------
((449, datetime.datetime(2022, 11, 3, 2, 10, 26), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F16%2F47%2F81%2F59%2F87%2FDSCRP_LO1647815987', 0, '로맨스 포맨 페로몬 남자향수 오드 퍼퓸 50ML', 58000, 'https://contents.lotteon.com/itemimage/_v194522/LO/16/47/81/59/87/_1/64/78/15/98/8/LO1647815987_1647815988_1.jpg/di

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v171234/LO/18/12/34/84/66/_1/81/23/48/46/7/LO1812348466_1812348467_2.jpg/dims/resizef/554X554
24
https://displayad.lotteon.com/cadClick?x=lX09d5CKBDqSrvrUEwdyl1JhdUpmBVWyL3r05WZ0hYqEGGvemEhfOy8rq9J%2B3QYBKO90MxuhmgPwASj1kRfnUO%2Fkcyhu%2FseEOGGPpBdkpNr%2B2c9R%2F%2B5vMF3G3UfHgGtTD5TBRHNyDKaAOFedCtTm5sgdC0%2FV%2BowH86oUeP4bT2UKIuA1kusDd1oftDU2NLVGOOKtoeLaC0Se8In1JphQ67O8Ug5A3PO9N8R9cNWAmheGN%2FqAzYX8ZOyEXsyLG2dPsA4BC9Wmd%2BbKNZMrDMjSrQ%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1516979276%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26dp_infw_cd%3DCASFC04140000
@@@@@@@@@@uid :  460
------------------------
((460, datetime.datetime(2022, 11, 3, 2, 12, 4), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F15%2F16%2F97%2F92%2F76%2FDSCRP_LO1516979276', 35, '클린 아크 앰플 쿠션 본품 리필 세트(구매시 리필 증정)', 54000, 'https://contents.lotteon.com/itemimage/_v161550/LO/15/16/97/92/76/_1/51/69/79/27/7/LO1516979276_

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v094939/LE/12/04/64/86/36/_1/22/51/40/89/2/LE1204648636_1225140892_1.jpg/dims/resizef/554X554
24
https://www.lotteon.com/p/product/LE1204576954?sitmNo=LE1204576954_1224688207&mall_no=1&dp_infw_cd=CASFC04140000
@@@@@@@@@@uid :  470
------------------------
((470, datetime.datetime(2022, 11, 3, 2, 17, 28), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLE%2F12%2F04%2F57%2F69%2F54%2FDSCRP_LE1204576954', 10, '래스팅 실크 UV 파운데이션 SPF20 (30ml)', 89000, 'https://contents.lotteon.com/itemimage/_v172910/LE/12/04/57/69/54/_1/22/46/88/20/7/LE1204576954_1224688207_1.jpg/dims/resizef/67x67', 4.8, '새상품', 183, 24, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v172910/LE/12/04/57/69/54/_1/22/46/88/20/7/LE1204576954_1224688207_1.jpg/dims/resizef/554X554
24
https://www.lotteon.com/p/product/LE1204369387?sitmNo=LE1204369387_1223472165&mall_no=1&dp_infw_cd=CASFC04140000
@@@@@@@@@@uid :  471
----------

((480, datetime.datetime(2022, 11, 3, 2, 19, 13), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F17%2F95%2F98%2F09%2F30%2FDSCRP_LO1795980930', 0, '(SK스토아 단독)페디 매직프레스 파라다이스 리조트 컬렉션', 61290, 'https://contents.lotteon.com/itemimage/_v195602/LO/17/95/98/09/30/_1/79/59/80/93/1/LO1795980930_1795980931_1.jpg/dims/resizef/67x67', 5.0, '새상품', 188, 25, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v195602/LO/17/95/98/09/30/_1/79/59/80/93/1/LO1795980930_1795980931_1.jpg/dims/resizef/554X554
25
https://displayad.lotteon.com/cadClick?x=DsWflGpe4Gz8N14q%2BLAMaDE2y9piG5mFd7eeuUR87Sw%2BjtiD%2F23FNTvpVW9DYhxROheHG1CGdgox5l3KejDcAeB7Z1o%2BVwxg7uiHylZPmu3K%2F4TkLxWTFuww7HfVhOOAQL5SHIyHhoKx2Vfa2oQhXymIjrOkVObu64ZsoK%2FVqn4VKDIHhb2hVQCXqs1b%2FPisySjitupOpTuqolKD1PP2DvfSXVzr%2FgeI7iiP%2BG%2B9oAP%2BPrUd7KUQxuGzza3Gx4HotDjfZ3fO%2BwNYc3klab0lwA%3D%3D&fwurl=https%3A%2F%2Fwww.lotteon.com%2Fp%2Fproduct%2FLO1862771712%3FareaCode%3DAD_CPC%26entryPoint%3Dad%26d

@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/LO/19/05/91/60/87/_1/90/59/16/08/8/LO1905916087_1905916088_1.jpg/dims/resizef/554X554
25
https://www.lotteon.com/p/product/LE1205031380?sitmNo=LE1205031380_1227428270&mall_no=1&dp_infw_cd=CASFC04020000
@@@@@@@@@@uid :  491
------------------------
((491, datetime.datetime(2022, 11, 3, 2, 22, 45), 3000, 'https://red.lotteon.com/goodsdetail?view=type1-raw&model=itemdetail%2FLO%2F12%2F05%2F56%2F73%2F35%2FDSCRP_LO1205567335', 0, '당일출고 SUN3 48W전문가 젤램프 2.0 스마트 UVLED겸용 네일국가고시', 39000, 'https://contents.lotteon.com/itemimage/_v152505/LO/12/05/56/73/35/_1/20/55/67/33/6/LO1205567335_1205567336_1.jpg/dims/resizef/67x67', 0.0, '새상품', 1, 25, None),)
@@@@@@@@@@메인썸네일 :  https://contents.lotteon.com/itemimage/_v112850/LE/12/05/03/13/80/_1/22/74/28/27/0/LE1205031380_1227428270_1.jpg/dims/resizef/554X554
25
https://www.lotteon.com/p/product/LO1968136476?sitmNo=LO1968136476_1968136477&mall_no=1&dp_infw_cd=CASFC04020000
@@@@@@@@@@uid :  492
-------

<function TextIOWrapper.close()>

# 📌 상품 정보 크롤링(마켓컬리)

In [44]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# 주소 담을 리스트
page_list = []

# small_categories.txt 파일 읽어오기
small_categories = "C:\SSAFY\자율프로젝트\자율크롤링\마켓컬리.txt"
# small_categories = "D:\SSAFY\자율\크롤링\자율크롤링\small_categories.txt"
f = open(small_categories, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = 2
    category_uid = str(category_uid)
    category_url = line
    print(category_uid)
    print(category_url)
    
    ## 크롤링
    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    webdriver_options.add_argument("headless")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(category_url)
    web_driver.implicitly_wait(5)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    link_list = soup.find_all('div', class_='css-1xyd46f',href=True)
    
#     print(link_list)
    # print(len(link_list))

#     목록에서 link 얻어오기 (2개씩 얻어와지기때문에 짝수번은 날려보내기)
    for index, list in enumerate(link_list):
        page_list.append(category_uid+" "+"https://www.kurly.com"+list['href']+ " " +list.find('img')['src'] )
            
time.sleep(5)

    # page_list 저장하기 
# fo = open("D:\SSAFY\자율\크롤링\자율크롤링\products.txt", "w", encoding="UTF8")
fo = open("C:\SSAFY\자율프로젝트\자율크롤링\마켓컬리상품.txt", "w", encoding="UTF8")
for page in page_list:
    print(page)
    fo.write(page)
    fo.write("\n")
fo.close
    
# print(len(page_list))

    
f.close


    




2
https://www.kurly.com/search?sword=%EC%83%8C%EB%93%9C%EC%9C%84%EC%B9%98&page=1

2
https://www.kurly.com/search?sword=%EC%83%8C%EB%93%9C%EC%9C%84%EC%B9%98&page=2

2
https://www.kurly.com/search?sword=%EC%83%8C%EB%93%9C%EC%9C%84%EC%B9%98&page=3
2 https://www.kurly.com/goods/5136646 https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/165224713535l0.jpg
2 https://www.kurly.com/goods/5041968 https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1648210421245l0.jpeg
2 https://www.kurly.com/goods/5042318 https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1572324205916l0.jpg
2 https://www.kurly.com/goods/5069168 https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1618564164328l0.jpg
2 https://www.kurly.com/goods/5052131 https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1637826749225l0.jpg
2 https://www.kurly.com/goods/1000029138 https://product-image.kurly.com/cdn-cgi

<function TextIOWrapper.close()>

### ✏️ 마켓컬리용 product, option insert 

In [48]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
import re
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from openpyxl import Workbook
import pymysql
from parse import *


def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn

products = "C:\SSAFY\자율프로젝트\자율크롤링\마켓컬리상품.txt"
f = open(products, "r", encoding="UTF8")


while True:
    line = f.readline()
#     print(line)
    if not line: break
    strings = line.split()
    category_uid = strings[0]
    url = strings[1]
    rep_img = strings[2]
    print(category_uid)
    print(url)
    print("------------------------")
    
    
    # 임시 url
#     temp_url= "https://www.kurly.com/goods/5136646"

    webdriver_options=webdriver.ChromeOptions()
    webdriver_options.add_argument("--start-maximized")
    webdriver_options.add_argument("headless")
    web_driver = webdriver.Chrome(options=webdriver_options)
    web_driver.get(url)
    web_driver.implicitly_wait(3)
    html = web_driver.page_source
    soup = BeautifulSoup(html, 'html.parser')


    # 데이터 찾기 
    product_name = soup.find('h1', class_='css-1f2zq3n').text

#     goods_intro = soup.find('div', class_='goods_intro').find('img')['src']
#     goods_point = soup.find('div', class_='goods_point').find('img')['src']
#     goods_desc = soup.find('div', class_='goods_desc').find('img')['src']
#     goods_pick = soup.find('div', class_='goods_pick').find('img')['src']



#     des_picture = str(goods_intro) + " "+ str(goods_point) + " "+str(goods_desc) + " "+str(goods_pick)
#     print(des_picture)
    
    status = 'SELLING'

    # 할인율이있는지 확인
    if soup.find_all("a", {"class": "staring"}):
        discount_rate = soup.find('span', class_='css-5nirzt').text.replace("%","")
        product_price = soup.find('span', class_='css-9pf1ze').text.replace(",","")
    else: # 없으면
        discount_rate = 0;
        product_price = soup.find('span', class_='css-9pf1ze').text.replace(",","")


    # int값으로 변환
    product_price = int(product_price)
    discount_rate = int(discount_rate)


    review_rating = 0
    # delivery_fee = soup.find_all('li', class_='soonInfoList')[1].text
    delivery_fee = 3000

    # 배송비 무료 > int 변환
    # val = "무료" in delivery_fee
    # if val:
    #     delivery_fee = 0
    # else:
    #     delivery_fee = 3000


#     rep_img = soup.find('div', class_='css-lmij30')
#     result = parse("background-image: url('{}');", rep_img['style'])[0] # :D



    ############### product insert ################
    # 형변환
    brand_uid = 1
    category_uid = 2
    category_uid = int(category_uid)
    review_rating = float(review_rating)

    print("delivery_fee",delivery_fee)
    print("discount_rate",discount_rate)
    print("price",product_price)
    print("name",product_name)
#     print("description_img",des_picture)
    print("rep_img",rep_img)
    print("review_rate",review_rating)
    print("status",status)
    print("brand_uid",brand_uid)
    print("small_categories_uid", category_uid)



    # print(type(delivery_fee))
    # print(type(des_picture))
    # print(type(discount_rate))
    # print(type(product_name))
    # print(type(product_price))
    # print(type(rep_img))
    # print(type(review_rating))
    # print(type(status))
    # print(type(brand_uid))
    # print(type(category_uid))


    sql = "INSERT INTO products (delivery_fee, discount_rate, name, price, rep_img, review_rate, status, brands_uid, small_categories_uid) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    with connectiondb() as conn:
        with conn.cursor() as cur:
            cur.execute(sql, (delivery_fee, discount_rate, product_name, product_price, rep_img, review_rating, status, brand_uid, category_uid))
            product_uid = cur.lastrowid
            conn.commit()

    print("@@product insert@@")

    option_list = ['불고기 샌드위치', '리코타 샌드위치', '튜나 샌드위치', '햄치즈 샌드위치']
    print(product_uid)
    
    for option in option_list:
        print(option)
        sql = "insert into wkdbf.product_options (name, value, products_uid) values (%s,%s,%s);"
        with connectiondb() as conn: 
            with conn.cursor() as cur:
                cur.execute(sql, ('종류', option, product_uid))
                conn.commit()

    print("@@option insert@@")

    print("====================================================")



f.close


2
https://www.kurly.com/goods/5041968
------------------------
delivery_fee 3000
discount_rate 0
price 4557
name [그린밤] 샐러드 샌드위치 6종
rep_img https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1648210421245l0.jpeg
review_rate 0.0
status SELLING
brand_uid 1
small_categories_uid 2
@@product insert@@
743
불고기 샌드위치
리코타 샌드위치
튜나 샌드위치
햄치즈 샌드위치
@@option insert@@
2
https://www.kurly.com/goods/5042318
------------------------
delivery_fee 3000
discount_rate 0
price 3900
name [쉐푸드] B.E.L.T 샌드위치 & 닭가슴살 흑임자 샌드위치
rep_img https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1572324205916l0.jpg
review_rate 0.0
status SELLING
brand_uid 1
small_categories_uid 2
@@product insert@@
744
불고기 샌드위치
리코타 샌드위치
튜나 샌드위치
햄치즈 샌드위치
@@option insert@@
2
https://www.kurly.com/goods/5069168
------------------------
delivery_fee 3000
discount_rate 0
price 4300
name [에브리데이] 크로와상 샌드위치 5종
rep_img https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1618564

튜나 샌드위치
햄치즈 샌드위치
@@option insert@@
2
https://www.kurly.com/goods/5131306
------------------------
delivery_fee 3000
discount_rate 0
price 5170
name [카페마마스] 햄치즈 샌드위치
rep_img https://img-cf.kurly.com/cdn-cgi/image/width=676,format=auto/shop/data/goods/1651219199994l0.jpg
review_rate 0.0
status SELLING
brand_uid 1
small_categories_uid 2
@@product insert@@
763
불고기 샌드위치
리코타 샌드위치
튜나 샌드위치
햄치즈 샌드위치
@@option insert@@
2
https://www.kurly.com/goods/1000043894
------------------------
delivery_fee 3000
discount_rate 0
price 5900
name [퍼스트클래스] 샌드위치 밀박스 4종 (택1)
rep_img https://product-image.kurly.com/cdn-cgi/image/width=676,format=auto/product/image/07c1514b-f3cd-4ae8-b108-513e77094128.jpg
review_rate 0.0
status SELLING
brand_uid 1
small_categories_uid 2
@@product insert@@
764
불고기 샌드위치
리코타 샌드위치
튜나 샌드위치
햄치즈 샌드위치
@@option insert@@
2
https://www.kurly.com/goods/5131307
------------------------
delivery_fee 3000
discount_rate 0
price 5170
name [카페마마스] 클럽 샌드위치
rep_img https://img-cf.kurly.com/cdn-cgi/ima

<function TextIOWrapper.close()>

### 인벤토리 추가

In [53]:
import pymysql

def connectiondb():
    conn = pymysql.connect(host='k7e201.p.ssafy.io',
                       user='abuyz7e201',
                       password='a-buy-z-e201',
                       db='wkdbf',
                       charset='utf8')
    return conn


# 743~ 779 / 746 제외
uid = 743

while True:
    print(uid)
    if(uid == 780): break
    
    sql = "select * from wkdbf.product_options where products_uid = %s"
    with connectiondb() as conn: 
        with conn.cursor() as cur:
            cur.execute(sql, uid)
            result = cur.fetchall()
            
    print("-----------------")
    for temp in result:
        print(temp[0])

        sql = "insert into wkdbf.inventories (count, price,product_option_list, products_uid) values (%s,%s,%s,%s);"
        with connectiondb() as conn: 
            with conn.cursor() as cur:
                cur.execute(sql, (100,0, temp[0], uid))
                conn.commit()
    
    print("====================")
    uid += 1
    if (uid == 746):
        uid += 1

743
-----------------
1107
1108
1109
1110
744
-----------------
1111
1112
1113
1114
745
-----------------
1115
1116
1117
1118
747
-----------------
1122
1123
1124
1125
748
-----------------
1126
1127
1128
1129
749
-----------------
1130
1131
1132
1133
750
-----------------
1134
1135
1136
1137
751
-----------------
1138
1139
1140
1141
752
-----------------
1142
1143
1144
1145
753
-----------------
1146
1147
1148
1149
754
-----------------
1150
1151
1152
1153
755
-----------------
1154
1155
1156
1157
756
-----------------
1158
1159
1160
1161
757
-----------------
1162
1163
1164
1165
758
-----------------
1166
1167
1168
1169
759
-----------------
1170
1171
1172
1173
760
-----------------
1174
1175
1176
1177
761
-----------------
1178
1179
1180
1181
762
-----------------
1182
1183
1184
1185
763
-----------------
1186
1187
1188
1189
764
-----------------
1190
1191
1192
1193
765
-----------------
1194
1195
1196
1197
766
-----------------
1198
1199
1200
1201
767
-----------------
1202
1203
12